In [2]:
import tkinter
import os
import re
import tkinter.filedialog
import InvertedIndex
from LinkedStack import LinkedStack
from BST import BST
from LinkedQueue import LinkedQueue
from TST import TST
from TrieST import TrieST

Stack = LinkedStack

global resultText
global files_list
global stopwordsTST
global stopwordsBST
global stopwordsTrie
global words_tree

main_command_line = tkinter.Entry
files_list = list()
tree_type_global = None
main_stack = Stack()
secondary_stack = Stack()
words_tree = None
directory_text_field_global = None
resultText = tkinter.Text()
directory_var = ""

stopwordsTST = TST()
stopwordsBST = BST()
stopwordsTrie = TrieST()
directory_var = ""


def ask_directory(text_label):
    folder = tkinter.filedialog.askdirectory(initialdir='/')
    text_label.delete(0, tkinter.END)
    text_label.insert(0, str(folder))
    return folder


def tab(arg):
    print("tab pressed")
    return 'break'


def write_result(inputStr):
    resultText.config(state=tkinter.NORMAL)
    resultText.insert(tkinter.INSERT, inputStr)
    resultText.config(state=tkinter.DISABLED)


def enter(arg):
    while not secondary_stack.is_empty():
        main_stack.push(secondary_stack.pop())
    command_line_content = main_command_line.get()
    main_stack.push(command_line_content)
    print(command_line_content)
    main_command_line.delete(0, tkinter.END)

    sytax_of_command_line(command_line_content)

    return 'break'


def reset():
    words_tree = None
    resultText.config(state=tkinter.NORMAL)
    resultText.delete('1.0', tkinter.END)
    resultText.config(state=tkinter.DISABLED)
    return 'break'


def callback(sv: tkinter.StringVar, event):
    if len(sv.get()) > 0 and sv.get()[-1] == '':
        if not main_stack.is_empty():
            event.delete(0, tkinter.END)
            secondary_stack.push(main_stack.pop())
            event.insert(0, secondary_stack.top())
        elif not secondary_stack.is_empty():
            event.delete(0, tkinter.END)
            event.insert(0, secondary_stack.top())
        else:
            event.delete(0, tkinter.END)
            event.insert(0, '')
    elif len(sv.get()) > 0 and sv.get()[-1] == '':
        if not secondary_stack.is_empty():
            event.delete(0, tkinter.END)
            main_stack.push(secondary_stack.pop())
            if secondary_stack.is_empty():
                event.insert(0, '')
            else:
                event.insert(0, secondary_stack.top())
        else:
            event.delete(0, tkinter.END)
            event.insert(0, '')


def stopwords_init():
    fileQueue = LinkedQueue()
    fp = open("StopWords.txt", '+r')
    for line in fp.readlines():
        key = (line.rstrip('\n'))
        fileQueue.enqueue(key)
    fp.close()

    # listing stop-words in TST
    # stopwordsTST
    i = 0
    for q in fileQueue:
        stopwordsTST.put(str(q.element), i)
        i += 1

    # listing stop-words in BST
    # stopwordsBST
    for q in fileQueue:
        stopwordsBST.insert(str(q.element))

    # listing stop-words in TrieST
    # stopwordsTrie
    i = 0
    for q in fileQueue:
        stopwordsTrie.put(str(q.element), i)
        i += 1


def build(directory_entered: tkinter.Entry, tree_type):
    global tree_type_global
    tree_type_global = tree_type
    if os.path.isdir(directory_entered.get()):
        if tree_type.get() == 1:
            # TST Tree
            words_tree = TST()
            i = 0
            for subdir, dirs, files in os.walk(directory_entered.get()):
                for file in files:
                    if file.endswith('.txt'):
                        fileQueue = LinkedQueue()
                        fp = open(os.path.join(subdir, file), 'r+', errors='ignore')
                        for line in fp.readlines():
                            key = (line.rstrip('\n'))
                        fileQueue.enqueue(key)
                        fp.close()
                        words_tree.get_valid_words(fileQueue)
                        files_list.append(fileQueue)

                        for q in fileQueue:
                            if stopwordsTST.get(q.element) == None:
                                words_tree.put(str(q.element), i)
                                i += 1

        elif tree_type.get() == 2:
            # BST Search
            words_tree = BST()
            for subdir, dirs, files in os.walk(directory_entered.get()):
                for file in files:
                    if file.endswith('.txt'):
                        fileQueue = LinkedQueue()
                        fp = open(os.path.join(subdir, file), 'r+', errors='ignore')
                        fileQueue = LinkedQueue()
                        for line in fp.readlines():
                            key = (line.rstrip('\n'))
                        fileQueue.enqueue(key)
                        fp.close()

                        files_list.append(fileQueue)
                        for q in fileQueue:
                            if stopwordsTST.get(q.element) == None:
                                words_tree.insert(str(q.element))

        elif tree_type.get() == 3:
            # Trie Search
            i = 0
            words_tree = TrieST()
            for subdir, dirs, files in os.walk(directory_entered.get()):
                for file in files:
                    if file.endswith('.txt'):
                        fileQueue = LinkedQueue()
                        fp = open(os.path.join(subdir, file), 'r+', errors='ignore')
                        fileQueue = LinkedQueue()
                        for line in fp.readlines():
                            key = (line.rstrip('\n'))
                        fileQueue.enqueue(key)
                        fp.close()

                        files_list.append(fileQueue)
                        for q in fileQueue:
                            if stopwordsTST.get(q.element) == None:
                                words_tree.put(str(q.element), i)
                                i += 1

    else:
        tkinter.messagebox.showinfo("Directory", "The Directory Entered doesn't Exist")


def sytax_of_command_line(command):
    # <-- It checks the sytax of input command by automata -->
    command_words = command.split()
    current_state = 0
    max_non_error_state = 15
    i = 0
    while i < len(command_words) and current_state < max_non_error_state:
        if current_state == 0:
            if command_words[0].lower() == 'add':
                current_state = 1
            elif command_words[0].lower() == 'del':
                current_state = 2
            elif command_words[0].lower() == 'update':
                current_state = 3
            elif command_words[0].lower() == 'list':
                current_state = 4
            elif command_words[0].lower() == 'search':
                current_state = 5
            else:
                current_state = 14
        elif current_state == 1:
            first_quote = re.match(r'^"(.*)', command_words[1])
            second_quote = re.match(r'(.*)"$', command_words[-1])
            if first_quote and second_quote:
                if command_words[1] == '\"':
                    del command_words[1]
                else:
                    command_words[1] = command_words[1].replace('\"', '')
                if command_words[-1] == '\"':
                    del command_words[-1]
                else:
                    command_words[-1] = command_words[-1].replace('\"', '')
                name_of_file = ''
                for separate_word in command_words[1:]:
                    name_of_file = name_of_file + separate_word
                file_exist = False
                for subdir, dirs, files in os.walk(directory_text_field_global.get()):
                    for file in files:
                        if name_of_file in file:
                            file_exist = True
                if [name_of_file_added for name_of_file_added in files_list if
                    name_of_file_added.documentName == name_of_file].__len__() > 0:
                    resultText.config(state=tkinter.NORMAL)
                    resultText.insert(tkinter.INSERT, 'Error : Document already Exists!!!\n---------------\n')
                    resultText.config(state=tkinter.DISABLED)
                elif not file_exist:
                    resultText.config(state=tkinter.NORMAL)
                    resultText.insert(tkinter.INSERT, 'Error : Document not Found!!!\n---------------\n')
                    resultText.config(state=tkinter.DISABLED)
                else:
                    build(directory_text_field_global, tree_type_global)

                    resultText.config(state=tkinter.NORMAL)
                    resultText.insert(tkinter.INSERT, 'File' + name_of_file + 'Added\n---------------\n')
                    resultText.config(state=tkinter.DISABLED)
            else:
                resultText.config(state=tkinter.NORMAL)
                resultText.insert(tkinter.INSERT, 'Error Happend')
                resultText.config(state=tkinter.DISABLED)
            return True
        elif current_state == 2:
            first_quote = re.match(r'^"(.*)', command_words[1])
            second_quote = re.match(r'(.*)"$', command_words[-1])
            if first_quote and second_quote:
                if command_words[1] == '\"':
                    del command_words[1]
                else:
                    command_words[1] = command_words[1].replace('\"', '')
                if command_words[-1] == '\"':
                    del command_words[-1]
                else:
                    command_words[-1] = command_words[-1].replace('\"', '')
                name_of_file = ''
                for separate_word in command_words[1:]:
                    name_of_file = name_of_file + separate_word
                file_name_found = False
                files_to_delete = [name_of_file_added for name_of_file_added in files_list if
                                   name_of_file_added.documentName == name_of_file]
                for file_going_to_delete in files_to_delete:
                    file_name_found = True
                    file_going_to_delete.removeAll()
                    files_list.remove(file_going_to_delete)
                    del file_going_to_delete
                    resultText.config(state=tkinter.NORMAL)
                    resultText.insert(tkinter.INSERT, 'File ' + name_of_file + ' Deleted\n---------------\n')
                    resultText.config(state=tkinter.DISABLED)
                if not file_name_found:
                    write_result(resultText.insert(tkinter.INSERT, 'Error : Document not Found!!!\n---------------\n'))
            else:
                resultText.config(state=tkinter.NORMAL)
                resultText.insert(tkinter.INSERT, 'Error Happend\n---------------\n')
                resultText.config(state=tkinter.DISABLED)

            return True
        elif current_state == 3:
            first_quote = re.match(r'^"(.*)', command_words[1])
            second_quote = re.match(r'(.*)"$', command_words[-1])
            if first_quote and second_quote:
                if not first_quote.group(1) == '\"':
                    command_words[1] = command_words[1].replace('\"', '')
                if not second_quote.group(1) == '\"':
                    command_words[-1] = command_words[-1].replace('\"', '')
            else:
                resultText.config(state=tkinter.NORMAL)
                resultText.insert(tkinter.INSERT, 'Error Happend\n---------------\n')
                resultText.config(state=tkinter.DISABLED)
            return True
        elif current_state == 4:
            if command_words[1] == '-w':
                current_state = 9
            elif command_words[1] == '-l':
                current_state = 10
            elif command_words[1] == '-f':
                current_state = 11
        elif current_state == 5:
            if command_words[1] == '-s':
                current_state = 12
            elif command_words[1] == '-w':
                current_state = 13
        elif current_state == 6:
            return True
        elif current_state == 7:
            return True
        elif current_state == 8:
            return True
        elif current_state == 9:
            write_result(words_tree.traverse_words_documents())
            return True
        elif current_state == 10:
            for file in files_list:
                write_result(file.documentName + ' ')
            write_result('\nNumber of listed Docs = ' + files_list.__len__().__str__() + '\n---------------\n')
            return True
        elif current_state == 11:
            number_of_files = 0
            for subdir, dirs, files in os.walk(directory_text_field_global.get()):
                for file in files:
                    if file.endswith('.txt'):
                        write_result(file[:-4] + ' ')
                        number_of_files += 1
            write_result('\nNumber of all Docs = ' + number_of_files.__str__() + '\n---------------\n')
            return True

        elif current_state == 12:
            first_quote = re.match(r'^"(.*)', command_words[2])
            print('1')
            second_quote = re.match(r'(.*)"$', command_words[-1])
            print('2')
            if first_quote and second_quote:
                print('3')
                if not first_quote.group(1) == '\"':
                    command_words[2] = command_words[2].replace('\"', '')
                print('4')
                if not second_quote.group(1) == '\"':
                    command_words[-1] = command_words[-1].replace('\"', '')
                print('5')
            else:
                resultText.config(state=tkinter.NORMAL)
                resultText.insert(tkinter.INSERT, 'Error Happend\n---------------\n')
                resultText.config(state=tkinter.DISABLED)
            if command_words[-1] is command_words[2]:
                if not words_tree.get(command_words[-1]):
                    write_result('\nAny word found !!!\n---------------\n')
                else:
                    write_result(words_tree.get(command_words[-1]).refrence.getAll())
            current_state = 20  # <-- This live has to change -->
            return True
        elif current_state == 13:
            first_quote = re.match(r'^"(.*)', command_words[2])
            second_quote = re.match(r'(.*)"$', command_words[-1])
            if first_quote and second_quote:
                if not first_quote.group(1) == '\"':
                    command_words[2] = command_words[2].replace('\"', '')
                if not second_quote.group(1) == '\"':
                    command_words[-1] = command_words[-1].replace('\"', '')
            else:
                resultText.config(state=tkinter.NORMAL)
                resultText.insert(tkinter.INSERT, 'Error Happend')
                resultText.config(state=tkinter.DISABLED)
            if command_words[-1] is command_words[2]:
                if not words_tree.get(command_words[-1]):
                    write_result('\nAny word found !!!\n')
                else:
                    write_result(words_tree.get(command_words[-1]).refrence.getAll())
            current_state = 20  # <-- This live has to change -->
            return True
        else:
            resultText.config(state=tkinter.NORMAL)
            resultText.insert(tkinter.INSERT, 'Error : Unkown Command\n')
            resultText.config(state=tkinter.DISABLED)
            return True
    return True


if __name__ == '__main__':

    directory_var = ""
    root = tkinter.Tk()
    frame = tkinter.Frame(root)
    root.resizable(width=False, height=False)
    root.geometry('1200x1500')
    frame.pack()

    directory_frame = tkinter.Frame(root, height=1200, width=1500)
    directory_frame.pack(side=tkinter.TOP)

    directory_label = tkinter.Label(directory_frame, font=("Courier", 20), text="Please Enter the Directory: ")
    directory_label.pack(side=tkinter.TOP)

    directory_text_field = tkinter.Entry(directory_frame, font=("Courier", 20), width=50, textvariable=directory_var)
    directory_text_field.pack(side=tkinter.LEFT)
    directory_text_field_global = directory_text_field

    brows_button = tkinter.Button(directory_frame, font=("Courier", 20), text="Browse",
                                  command=lambda: ask_directory(directory_text_field))
    brows_button.pack(side=tkinter.RIGHT)

    result_frame = tkinter.LabelFrame(root, font=("Courier", 20), text='Result:')
    result_frame.pack()

    text = tkinter.Text(result_frame, font=("Courier", 20), height=40, width=70)
    scroll = tkinter.Scrollbar(result_frame, command=text.yview)
    text.pack()

    text.configure(yscrollcommand=scroll.set)
    text.config(state=tkinter.DISABLED)
    resultText = text

    search_options = tkinter.LabelFrame(root, font=("Courier", 20), text='choose Data Structure:')
    search_options.pack()

    var = tkinter.IntVar()

    rad1 = tkinter.Radiobutton(search_options, font=("Courier", 20), text="TST", variable=var, value=1)
    rad1.pack(side=tkinter.LEFT)
    rad1.invoke()

    rad2 = tkinter.Radiobutton(search_options, font=("Courier", 20), text="BST", variable=var, value=2)
    rad2.pack(side=tkinter.LEFT)

    rad3 = tkinter.Radiobutton(search_options, font=("Courier", 20), text="Trie", variable=var, value=3)
    rad3.pack(side=tkinter.LEFT)

    command_line_frame = tkinter.LabelFrame(root, font=("Courier", 20), text='Command Line : ')
    command_line_frame.pack()

    sv = tkinter.StringVar()
    command_line = tkinter.Entry(command_line_frame, font=("Courier", 20), width=40, textvariable=sv)
    sv.trace("w", lambda name, index, mode, sv=sv: callback(sv, command_line))
    command_line.bind('<Tab>', tab)
    command_line.bind('<Return>', enter)
    command_line.pack(side=tkinter.RIGHT)
    main_command_line = command_line

    buttons_frame = tkinter.Frame(root)
    buttons_frame.pack()

    build_button = tkinter.Button(buttons_frame, font=("Courier", 20), text="Build",
                                  command=lambda: build(directory_text_field, var))
    build_button.pack(side=tkinter.LEFT)

    build_button = tkinter.Button(buttons_frame, font=("Courier", 20), text="Reset", command=lambda: reset())
    build_button.pack(side=tkinter.LEFT)

    build_button = tkinter.Button(buttons_frame, font=("Courier", 20), text="Exit", command=lambda: root.destroy())
    build_button.pack(side=tkinter.RIGHT)

    build_button = tkinter.Button(buttons_frame, font=("Courier", 20), text="Help",
                                  command=lambda: ask_directory(directory_text_field))
    build_button.pack(side=tkinter.RIGHT)

    copy_right_frame = tkinter.Frame(root)
    copy_right_frame.pack(side=tkinter.BOTTOM)

    copy_right_lable1 = tkinter.Label(copy_right_frame, font=("Courier", 20), text="powerd by ")
    copy_right_lable2 = tkinter.Label(copy_right_frame, font=("Courier", 30), text="Mohammad Hossein Forouhesh Tehrani")

    copy_right_lable1.pack(side=tkinter.TOP)
    copy_right_lable2.pack(side=tkinter.BOTTOM)

    stopwordsTST = TrieST()
    fileQueue = LinkedQueue()
    fp = open("StopWords.txt", '+r')
    for line in fp.readlines():
        key = (line.rstrip('\n'))
        fileQueue.enqueue(key)
    fp.close()
    i = 0
    for q in fileQueue:
        stopwordsTrie.put(str(q.element), i)
        i += 1
    # s = stopwordsTST.get('about').refrence.getAll()
    # write_result(s)
    # write_result(stopwordsTST.traverse())

    root.mainloop()


ImportError: No module named 'InvertedIndex'